In [19]:
pip install langchain-core langgraph groq langchain-groq pypdf langchain-community

In [20]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all fastembed langchain-groq

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
import numpy as np
from typing import List, Dict
from langchain_core.tools import tool

embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
print(f"len of documents :{len(docs_list)}")

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)
print(f"length of document chunks generated :{len(doc_splits)}")

class VectorStoreRetriever:
    def __init__(self, docs: List[dict], vectors: List[List[float]], embed_model):
        self._arr = np.array(vectors)
        self._docs = docs
        self._embed_model = embed_model

    @classmethod
    def from_docs(cls, docs: List[dict], embed_model):
        texts = [doc.page_content for doc in docs]
        vectors = list(embed_model.embed_documents(texts))
        return cls(docs, vectors, embed_model)

    def query(self, query: str, k: int = 5) -> List[Dict]:
      query_vector = self._embed_model.embed_query(query)
      scores = np.array(query_vector) @ self._arr.T
      top_k_idx = np.argpartition(scores, -k)[-k:]
      top_k_idx_sorted = top_k_idx[np.argsort(-scores[top_k_idx])]
      return [
          {
              "page_content": self._docs[idx].page_content,
              "metadata": self._docs[idx].metadata,
              "similarity": scores[idx]
          }
          for idx in top_k_idx_sorted
      ]


retriever = VectorStoreRetriever.from_docs(doc_splits, embed_model)


@tool
def lookup_policy(query: str) -> str:
    """Consult the company policies to check whether certain options are permitted.
    Use this before making any flight changes performing other 'write' events."""
    docs = retriever.query(query, k=2)
    return "\n\n".join([doc["page_content"] for doc in docs])


len of documents :3
length of document chunks generated :87


In [62]:
# import re
# import numpy as np
# import requests
# from langchain_core.tools import tool
# from sentence_transformers import SentenceTransformer

# # Load FAQ markdown
# response = requests.get(
#     "https://storage.googleapis.com/benchmarks-artifacts/travel-db/swiss_faq.md"
# )
# response.raise_for_status()
# faq_text = response.text

# docs = [{"page_content": txt} for txt in re.split(r"(?=\n##)", faq_text, flags=re.MULTILINE)]

# # docs = [{"page_content": chunk.strip()} for chunk in re.split(r"(?=^## )", faq_text, flags=re.MULTILINE) if chunk.strip()]

# # Load a Hugging Face embedding model
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


# class VectorStoreRetriever:
#     def __init__(self, docs: list, vectors: list):
#         self._arr = np.array(vectors)
#         self._docs = docs

#     ###@classmethod
#     def from_docs(cls, docs):
#         texts = [doc["page_content"] for doc in docs]
#         vectors = model.encode(texts, normalize_embeddings=False)
#         return cls(docs, vectors)

#     def query(self, query: str, k: int = 5) -> list[dict]:
#         query_vec = model.encode([query], normalize_embeddings=False)[0]
#         scores = query_vec @ self._arr.T  # cosine similarity
#         top_k_idx = np.argpartition(scores, -k)[-k:]
#         top_k_idx_sorted = top_k_idx[np.argsort(-scores[top_k_idx])]
#         return [
#             {**self._docs[idx], "similarity": float(scores[idx])} for idx in top_k_idx_sorted
#         ]


# retriever = VectorStoreRetriever.from_docs(docs)


# ###@tool
# def lookup_policy(query: str) -> str:
#     """Consult the company policies to check whether certain options are permitted.
#     Use this before making any flight changes or performing other 'write' events."""
#     docs = retriever.query(query, k=2)
#     return "\n\n".join([doc["page_content"] for doc in docs])

In [22]:
lookup_policy.invoke("What are agents?")

'Planning is essentially in order to optimize believability at the moment vs in time.\nPrompt template: {Intro of an agent X}. Here is X\'s plan today in broad strokes: 1)\nRelationships between agents and observations of one agent by another are all taken into consideration for planning and reacting.\nEnvironment information is present in a tree structure.\n\n\n\n\nFig. 13. The generative agent architecture. (Image source: Park et al. 2023)\nThis fun simulation results in emergent social behavior, such as information diffusion, relationship memory (e.g. two agents continuing the conversation topic) and coordination of social events (e.g. host a party and invite many others).\nProof-of-Concept Examples#\nAutoGPT has drawn a lot of attention into the possibility of setting up autonomous agents with LLM as the main controller. It has quite a lot of reliability issues given the natural language interface, but nevertheless a cool proof-of-concept demo. A lot of code in AutoGPT is about for

In [29]:
import os
import json  # Added missing import
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage  # Consolidated imports
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from typing import Annotated, List  # More specific typing
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from google.colab import userdata
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode, tools_condition


# Initialize LLM
llm = ChatGroq(
    api_key=userdata.get("groq_api_key"),
    model="llama3-70b-8192",  # Changed to more reliable model
    temperature=0.7,
)

# Initialize memory
memory = MemorySaver()

tools = [lookup_policy]
llm_with_tools = llm.bind_tools(tools)

class State(TypedDict):
    messages: Annotated[List[dict], add_messages]  # More specific type

In [35]:
from datetime import date, datetime
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful customer support assistant for Swiss Airlines. "
            " Use the provided tools to search for flights, company policies, and other information to assist the user's queries. "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            " If a search comes up empty, expand your search before giving up."
            "\n\nCurrent user:\n<User>\n{user_info}\n</User>"
            "\nCurrent time: {time}.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)


def chatbot(state: dict):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}



# Fix: Pass the actual tool list, not a single undefined 'tool'
tool_node = ToolNode(tools=[lookup_policy])

def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END

# Graph setup
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    {"tools": "tools", END: END},
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

# Compile the graph
graph = graph_builder.compile(checkpointer=memory)

def stream_graph_updates(user_input: str, i:int):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}, {"configurable": {"thread_id": str(i)}}):
        for value in event.values():
          if ((value["messages"][-1].content != '') or (value["messages"][-1].tool_call_id.length)):
            print("Assistant:", value["messages"][-1].content)

# Main conversation loop
print("""
Welcome to the MCP Analysis Tool!
I can answer questions about companies based on the provided documents.
Type 'quit', 'exit', or 'q' to end the conversation.
""")

thread_id = 203
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            thread_id+=1
            break
        stream_graph_updates(user_input, thread_id)
    except:
        # fallback if input() is not available
        print("Try failed, on the fallback now")
        break


Welcome to the MCP Analysis Tool!
I can answer questions about companies based on the provided documents.
Type 'quit', 'exit', or 'q' to end the conversation.

User: Hi
Assistant: Hi!
User: Why this is failing sometimes?
Try failed, on the fallback now
